### Imports

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# 

### Model / data parameters

In [2]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [16]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(4, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(4, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 4)         40        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 4)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 4)         148       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 4)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1010      
Total params: 1,198
Trainable params: 1,198
Non-trainable params: 0
____________________________________________________

# 

### Train the model

In [17]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 10s 23ms/step - loss: 0.8885 - accuracy: 0.7280 - val_loss: 0.2501 - val_accuracy: 0.9265
Epoch 2/15
422/422 [==============================] - 9s 22ms/step - loss: 0.2654 - accuracy: 0.9179 - val_loss: 0.1840 - val_accuracy: 0.9467
Epoch 3/15
422/422 [==============================] - 10s 24ms/step - loss: 0.2091 - accuracy: 0.9362 - val_loss: 0.1557 - val_accuracy: 0.9565
Epoch 4/15
422/422 [==============================] - 10s 23ms/step - loss: 0.1819 - accuracy: 0.9445 - val_loss: 0.1380 - val_accuracy: 0.9602
Epoch 5/15
422/422 [==============================] - 10s 24ms/step - loss: 0.1657 - accuracy: 0.9485 - val_loss: 0.1339 - val_accuracy: 0.9615
Epoch 6/15
422/422 [==============================] - 9s 22ms/step - loss: 0.1537 - accuracy: 0.9523 - val_loss: 0.1227 - val_accuracy: 0.9650
Epoch 7/15
422/422 [==============================] - 9s 22ms/step - loss: 0.1444 - accuracy: 0.9552 - val_loss: 0.1244 - val_accuracy: 0.

# 

### Evaluate the trained model

In [18]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.09349916875362396
Test accuracy: 0.9710999727249146


# 

### Convert model to tflite and save it in C format

In [19]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/l2/590l7mh12zs5c8p1xj5r8h8h0000gn/T/tmp74_6krjr/assets


INFO:tensorflow:Assets written to: /var/folders/l2/590l7mh12zs5c8p1xj5r8h8h0000gn/T/tmp74_6krjr/assets


In [20]:
from tensorflow.lite.python.util import convert_bytes_to_c_source


source_text, header_text = convert_bytes_to_c_source(tflite_model,  "mnist_tflite")

with  open('inc/mnist_tflite.h',  'w')  as  file:
    file.write(header_text)

with  open('src/mnist_tflite.cc',  'w')  as  file:
    file.write(source_text)